# This notebook creates a coordiantion-aware community graph for a big cluster

The dash implementation has the problem, that it runs out of memory if too many connections are in a network.
Thus, for the Swift case the coordination-aware community graph for the 60% coordination level was created with this notebook.

### Handle imports

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from coordination_detection import pipeline

import pandas as pd
from community import community_louvain
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.cm as cm

### Load Twitter API access secrets

In [ ]:
%load_ext dotenv
%dotenv

### Calculate the similarity with the new coordination pipeline

In [4]:
similarity = pipeline.calculate_coordination_pipeline_twitter_data("8273444c-abdd-4410-829a-970846ebd00e", 34859, "2022-02-25T14:07:13")

100%|██████████| 3000/3000 [16:05<00:00,  3.11it/s]   


Spawn two processes


### Save the result for the case that the notebook kernel crashes

In [4]:
similarity.to_parquet("./parquet_saves/swift twitter data.snappy", compression="snappy")

### Load the results from parquet

In [ ]:
similarity = pd.read_parquet("./parquet_saves/swift twitter data.snappy")

### Filter the similarities for the 60\% coordination level

In [8]:
similarities = similarity[similarity["Weight"] > 0.6]

### Plot the graph

In [10]:
edge_labels = {(row["User1"], row["User2"]): {"Method": row["Method"], "Weight": row["Weight"]} for _, row in similarities.iterrows()}

In [ ]:
%matplotlib inline
similarities = similarity[similarity["Weight"] > 0.6]
G = nx.from_pandas_edgelist(similarities, "User1", "User2", edge_attr=["Weight", "Method"], create_using=nx.MultiGraph())
pos = nx.spring_layout(G)
partition = community_louvain.best_partition(G, weight="Weight")
cmap = cm.get_cmap('viridis', max(partition.values()) + 1)
fig, ax = plt.subplots(figsize=(15, 10), dpi=60)
nx.draw_networkx_nodes(G, pos, partition.keys(), node_size=150, linewidths=2, edgecolors="#000", 
                       cmap=cmap, node_color=list(partition.values()), ax=ax)
edges = nx.draw_networkx_edges(G, pos, alpha=1.0, edge_color="#888", width=0.5, ax=ax)

annot = ax.annotate("", xy=(0,0), xytext=(20,20),textcoords="offset points",
                    bbox=dict(boxstyle="round", fc="w"),
                    arrowprops=dict(arrowstyle="->"))
annot.set_visible(False)

ax.set_facecolor("#E5ECF6")

def update_annot(ind):
    edge = list(G.edges)[ind["ind"][0]]
    xy = (pos[edge[0]] + pos[edge[1]])/2
    annot.xy = xy
    node_attr = {'edge': edge}
    node_attr.update(G.edges[edge])
    text = '\n'.join(f'{k}: {v}' for k, v in node_attr.items())
    annot.set_text(text)

def hover(event):
    vis = annot.get_visible()
    if event.inaxes == ax:
        cont, ind = edges.contains(event)
        if cont:
            update_annot(ind)
            annot.set_visible(True)
            fig.canvas.draw_idle()
        else:
            if vis:
                annot.set_visible(False)
                fig.canvas.draw_idle()

fig.canvas.mpl_connect("motion_notify_event", hover)

plt.show()